In [1]:
import os
import sys
import json
import time
import base64
from PIL import Image
import io
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv, set_key
from src.encrypt import decrypt_data
# from src.encrypt import env_file
from src.auth_helpers import get_access_token   
from utils.chat_model import AIGatewayLangchainChatOpenAI

In [2]:
with open('secret/url.json', 'r') as file:
    url_data = json.load(file)
AI_GATEWAY_BASE_URL = url_data.get("ai_gateway")
issuer_url = url_data.get("issuer_url")

# Load the .env file
load_dotenv("secret/.env")

# Retrieve the Fernet key and encrypted values
loaded_fernet_key = os.getenv('FERNET_KEY').encode()
loaded_encrypted_client_id = os.getenv('ENCRYPTED_CLIENT_ID').encode()
loaded_encrypted_client_secret = os.getenv('ENCRYPTED_CLIENT_SECRET').encode()

# Decrypt the values
client_id = decrypt_data(loaded_encrypted_client_id, loaded_fernet_key)
client_secret = decrypt_data(loaded_encrypted_client_secret, loaded_fernet_key)

In [3]:
access_token = get_access_token(client_id, client_secret, issuer_url)
# print(access_token)

In [4]:
# Initialize the language model with the access token and base URL
model = AIGatewayLangchainChatOpenAI(
    access_token=access_token, base_url=AI_GATEWAY_BASE_URL, model="gpt-4o-2024-05-13", deere_ai_gateway_registration_id="graphics-quality-check")

In [5]:
def invoke_model(image_base64, prompt):
    message = HumanMessage(
        content=[
            {"type": "text", "text": prompt},
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image_base64}"},
            },
        ],
    )
    start_time = time.time()
    response = model.invoke([message])
    end_time = time.time()
    return response.content

In [6]:
def convert_image(image_path):
        image = Image.open(image_path)

        # Ensure the image is in a supported format
        if image.format not in ["PNG", "JPEG", "GIF", "WEBP"]:
            print(f"Unsupported image format: {image.format}")
            return
        else:
            # Convert the image to base64
            buffered = io.BytesIO()
            image.save(buffered, format=image.format)
            image_data = buffered.getvalue()
            image_base64 = base64.b64encode(image_data).decode("utf-8")
            return image_base64

In [7]:
im_path = r"C:\Users\CDKVF7A\OneDrive - Deere & Co\Graphics_Quality_Check\graphics\graphics\25.png"

In [8]:
im_bas64 = convert_image(im_path)

In [9]:
prompt = '''Extract the following parameters from the provided graphics in json format along with parameter and value:
        1. callout
        2. dpi
        3. Callout Font
        4. Image Size '''

In [10]:
result = invoke_model(im_bas64, prompt)
print(result)

```json
{
  "callout": ["B", "C", "D"],
  "dpi": 72,
  "Callout Font": "Arial",
  "Image Size": "225 x 135 pixels"
}
```


In [11]:
# prompt = "check if based on GD&T standards datums are present accurately in the image or not."